In [1]:
#!git clone https://github.com/tylerachang/word-acquisition-language-models

In [2]:
!pip install numpy
!pip install scipy
!pip install torch
!pip install nltk
!pip install sentencepiece
!pip install tokenizers
!pip install transformers[torch]
!pip install spacy
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.6 MB/s eta 0:00:00


In [3]:
!pip install git+https://github.com/UIUCLearningLanguageLab/AOCHILDES

  Cloning https://github.com/UIUCLearningLanguageLab/AOCHILDES to /tmp/pip-req-build-5alek4wk
  Running command git clone --filter=blob:none --quiet https://github.com/UIUCLearningLanguageLab/AOCHILDES /tmp/pip-req-build-5alek4wk
  Resolved https://github.com/UIUCLearningLanguageLab/AOCHILDES to commit e25348138282c35f4270800e592676b8713e67a1
  Preparing metadata (setup.py) ... done
  Created wheel for aochildes: filename=aochildes-3.0.0-py3-none-any.whl size=41322875 sha256=38cc126d679a6d8339019ddf3c47c52d8ee83744eb409ea9a4bbe5e456f3e6e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-5ywborox/wheels/b7/9e/c5/cd013d6d40aa0309899f3232a4779e769b222d7bec7e8724d3
Successfully built aochildes


In [51]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopword_set = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from aochildes.dataset import AOChildesDataSet
transcripts = AOChildesDataSet().load_transcripts()

In [5]:
import re
for i, _ in enumerate(transcripts):
    transcripts[i] = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", transcripts[i]) # removing punctuation
    transcripts[i] = " ".join(transcripts[i].split()) # removing multiple spaces
    transcripts[i] = " " + transcripts[i]

In [48]:
import numpy as np
lens = np.array([len(t) for t in transcripts])
print(lens.mean())

5874.176150121066


In [49]:
print(len(transcripts))

3304


In [8]:
transcripts[0]

' just like your book at home do you want to look at that it says look see theres a face with glasses aha look theres pooh bear theres a baby bear with his bottle whos that is that child aha i think i see child there he is there he is yeah yeah there he is you see yourself thats my pretty baby boy im sorry thats handsome baby boy you like that yeah look theres a bunny rabbit see theres kitty cat oh kitty cat and whos this its a baby its a baby yeah want to look at that oh does it fit your mouth lets see what else there is in here theres a a pink pig oink you like that book its very red red is a good thing isnt it oink oink oink oink oink oink you want your book back dont you lets see do you want to see who this is this is big bird enough with that can you read your book again you like the book look see you want to see if it fits in your mouth is that what you want to see oh thats too big what do you know you want to hold it like that whoop here theres baby bear with his bottle see baby

In [9]:
import pandas as pd

In [11]:
aoa_df = pd.read_csv('bilstm_aoa.txt', sep='\t')

In [12]:
word_aoa = {}

In [13]:
for i, row in aoa_df.iterrows():
    word_aoa[row['Token']] = row['LmAoA']

In [62]:
def get_weighted_aoa(sentence, word_aoa):
    s = 0
    count = 0
    words = sentence.split(' ')
    for word in words:
        if word == ' ': #or word in stopword_set:
            continue
        if word in word_aoa:
            s += word_aoa[word]
            count += 1
    if count == 0:
        print("Div by 0")
        return 0
    return s / count

def get_max_aoa(sentence, word_aoa):
    mx = 0
    words = sentence.split(' ')
    for word in words:
        if word == ' ':
            continue
        if word in word_aoa:
            mx = max(mx, word_aoa[word])
    return mx


get_weighted_aoa(transcripts[0], word_aoa)

3.2989990882293414

In [63]:
curriculum_arr = []

for i, sentence in enumerate(transcripts):
    curriculum_arr.append((sentence, get_weighted_aoa(sentence, word_aoa), i))

Div by 0
Div by 0
Div by 0
Div by 0
Div by 0
Div by 0
Div by 0
Div by 0
Div by 0
Div by 0


In [64]:
curriculum_arr.sort(key=lambda s: -s[1])

curriculum = list(map(lambda x: x[0], curriculum_arr))
ordering = list(map(lambda x: x[2], curriculum_arr))

In [65]:
from scipy.stats import spearmanr

In [66]:
spearmanr(list(range(len(transcripts))), ordering)

SignificanceResult(statistic=0.5495004897321927, pvalue=4.550938526026888e-260)

In [59]:
def save_curriculum(curr, save_path):
    df = pd.DataFrame()
    df['curriculum'] = curr
    df.to_csv(save_path, index=False)

In [60]:
save_curriculum(transcripts, 'ao_childes_curriculum.csv')

In [67]:
#save_curriculum(curriculum, 'weighted_aoa_with_stopwords_curriculum.csv')
save_curriculum(curriculum, 'weighted_aoa_with_stopwords_curriculum.csv')

In [ ]:
# !rm -r sample_data
# !mkdir sample_data

mkdir: cannot create directory ‘sample_data’: File exists
